In [1]:
import pandas as pd

In [59]:
debt = pd.read_csv('debt.csv')

In [60]:
debt.rename(columns={'2019 [YR2019]': '2019', '2020 [YR2020]': '2020', '2021 [YR2021]': '2021', '2022 [YR2022]': '2022', '2023 [YR2023]': '2023'}, inplace=True)

In [61]:
debt.dropna(inplace=True)

In [62]:
debt['Country Name'] = debt['Country Name'].str.strip()
debt['Counterpart-Area Name'] = debt['Counterpart-Area Name'].str.strip()

In [63]:
debt = debt[(debt['2019'] != '..') & (debt['2020'] != '..') & (debt['2021'] != '..') & (debt['2022'] != '..') & (debt['2023'] != '..')]

In [64]:
debt

,Country Name,Country Code,Counterpart-Area Name,Counterpart-Area Code,Series Name,Series Code,2019,2020,2021,2022,2023
21,Afghanistan,AFG,Asian Dev. Bank,915,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,14275000,14273000,34522930.3,34120750.3,33717130.4
44,Afghanistan,AFG,Bulgaria,072,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,1000000,500000,1036250,1150107.1,1120107.1
142,Afghanistan,AFG,International Monetary Fund,907,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,11886621.3,9823909.6,5184966.5,5401006.7,4536845.7
146,Afghanistan,AFG,Islamic Dev. Bank,976,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,3707056.2,3518887.7,4039801.5,3979310.1,3917378.6
159,Afghanistan,AFG,Kuwait,552,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,111985.4,55514.4,111989.5,111989.5,111989.5
...,...,...,...,...,...,...,...,...,...,...,...
20573,Zambia,ZMB,United Arab Emirates,576,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,0,0,49012.9,49012.9,49012.9
20574,Zambia,ZMB,United Kingdom,012,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,227623000,183000000,392984144.5,361512862.4,73189747
20575,Zambia,ZMB,United States,302,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,4503000,5180000,9828800,9669800,9510800
20594,Zambia,ZMB,World,WLD,"Debt service on external debt, total (TDS, cur...",DT.TDS.DECT.CD,2604670780.3,4059774261.9,5903814690,5374664988.5,3743347273.8


In [65]:
debt = debt.melt(id_vars=['Country Name', 'Country Code', 'Counterpart-Area Name'], value_vars=['2019', '2020', '2021', '2022', '2023'], var_name='Year', value_name='Debt')

In [66]:
debt['Debt'] = debt['Debt'].astype(float)

In [67]:
debt

,Country Name,Country Code,Counterpart-Area Name,Year,Debt
0,Afghanistan,AFG,Asian Dev. Bank,2019,1.427500e+07
1,Afghanistan,AFG,Bulgaria,2019,1.000000e+06
2,Afghanistan,AFG,International Monetary Fund,2019,1.188662e+07
3,Afghanistan,AFG,Islamic Dev. Bank,2019,3.707056e+06
4,Afghanistan,AFG,Kuwait,2019,1.119854e+05
...,...,...,...,...,...
5840,Zambia,ZMB,United Arab Emirates,2023,4.901290e+04
5841,Zambia,ZMB,United Kingdom,2023,7.318975e+07
5842,Zambia,ZMB,United States,2023,9.510800e+06
5843,Zambia,ZMB,World,2023,3.743347e+09


In [68]:
pd.DataFrame(debt.groupby('Counterpart-Area Name')['Debt'].sum().sort_values(ascending=False))

,Debt
Counterpart-Area Name,
World,4.316449e+11
Other Multiple Lenders,1.671310e+11
China,6.572703e+10
Bondholders,4.959060e+10
World Bank-IDA,2.081378e+10
...,...
Grenada,4.588891e+05
St. Lucia,3.034261e+05
International Bank for Economic Cooperation (IBEC),1.855000e+05


In [89]:
grouped = debt.groupby(['Counterpart-Area Name', 'Country Name'])[['Debt']].sum()

import json
levels = len(grouped.index.levels)
dicts = [{} for i in range(levels)]
last_index = None

for index,value in grouped.itertuples():

    if not last_index:
        last_index = index

    for (ii,(i,j)) in enumerate(zip(index, last_index)):
        if not i == j:
            ii = levels - ii -1
            dicts[:ii] =  [{} for _ in dicts[:ii]]
            break

    for i, key in enumerate(reversed(index)):
        dicts[i][key] = value
        value = dicts[i]

    last_index = index


result = dicts[-1]
# result = json.dumps(dicts[-1])

debt_restructured = {'name': 'debt', 'children': []}

for i in result:
    obj = {'name': i, 'children': []}
    for j in result[i]:
        obj['children'].append({'name': j, 'value': result[i][j]})
    debt_restructured['children'].append(obj)
print(debt_restructured)

with open('debt.json', 'w') as f:
    f.write(json.dumps(debt_restructured, indent=4))

{'name': 'debt', 'children': [{'name': 'African Dev. Bank', 'children': [{'name': 'Angola', 'value': 480012272.1}, {'name': 'Benin', 'value': 63970863.2}, {'name': 'Burkina Faso', 'value': 77683143.5}, {'name': 'Burundi', 'value': 5705297.2}, {'name': 'Cabo Verde', 'value': 84108637.60000001}, {'name': 'Cameroon', 'value': 160923862.9}, {'name': 'Central African Republic', 'value': 254365.9}, {'name': 'Chad', 'value': 19701006.1}, {'name': 'Comoros', 'value': 93925.0}, {'name': 'Congo, Dem. Rep.', 'value': 497863754.3}, {'name': 'Congo, Rep.', 'value': 46652196.300000004}, {'name': "Cote d'Ivoire", 'value': 50753878.2}, {'name': 'Djibouti', 'value': 26634170.4}, {'name': 'Ethiopia', 'value': 287761508.6}, {'name': 'Gambia, The', 'value': 12669069.4}, {'name': 'Ghana', 'value': 197701632.2}, {'name': 'Guinea', 'value': 22089568.8}, {'name': 'Guinea-Bissau', 'value': 4967443.6}, {'name': 'Kenya', 'value': 369996436.4}, {'name': 'Lesotho', 'value': 39753638.5}, {'name': 'Liberia', 'value'

In [88]:
dicts[-1]

{'African Dev. Bank': {'Angola': 480012272.1,
  'Benin': 63970863.2,
  'Burkina Faso': 77683143.5,
  'Burundi': 5705297.2,
  'Cabo Verde': 84108637.60000001,
  'Cameroon': 160923862.9,
  'Central African Republic': 254365.9,
  'Chad': 19701006.1,
  'Comoros': 93925.0,
  'Congo, Dem. Rep.': 497863754.3,
  'Congo, Rep.': 46652196.300000004,
  "Cote d'Ivoire": 50753878.2,
  'Djibouti': 26634170.4,
  'Ethiopia': 287761508.6,
  'Gambia, The': 12669069.4,
  'Ghana': 197701632.2,
  'Guinea': 22089568.8,
  'Guinea-Bissau': 4967443.6,
  'Kenya': 369996436.4,
  'Lesotho': 39753638.5,
  'Liberia': 9551538.6,
  'Madagascar': 58520864.8,
  'Malawi': 45424307.6,
  'Mali': 96907903.7,
  'Mauritania': 113383014.2,
  'Mozambique': 123570353.9,
  'Niger': 41502394.5,
  'Nigeria': 647229998.5,
  'Rwanda': 75207419.19999999,
  'Sao Tome and Principe': 5385625.4,
  'Senegal': 153856646.5,
  'Sierra Leone': 17468154.099999998,
  'Somalia': 119135626.39999999,
  'Tanzania': 270621191.1,
  'Togo': 4508653.2,


In [17]:
debt.to_csv('debt_clean.csv', index=False)